In [0]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.special 
from sympy.integrals.quadrature import gauss_lobatto
from sklearn.utils import shuffle

In [0]:
from IPython.display import clear_output
import timeit

# Nodi di interpolazione

Il seguente codice costruisce i nodi di interpolazione sull'elemento di riferimento $\hat{K}=[-1,1]^d$.

La funzione che ci servirà sarà **generate_interpolation_nodes** che vuole tre argomenti:

- n : numero di punti di Chebyshev-Gauss-Lobatto che andranno poi resi una griglia;

- d : dimensione della griglia (d = 1, 2 o 3);

- plot (default = False): se è True, la griglia generata verrà plottata.

La funzione ritorna un np.array di dimensione $(n^d,d)$.

In [0]:
def generate_interpolation_nodes(n, d):
    """
    Restituisce i nodi di interpolazione per l'elemento di riferimento in d dimensioni
    """
    assert d==1 or d==2 or d==3
    assert n>1

    def generate_CGL_nodes(n):
        q, w = gauss_lobatto(n, 10)
        q = np.array(q, dtype=float)
        return q

    def generate_interpolation_nodes_2d(n):
        q = generate_CGL_nodes(n)
        Q1, Q2 = np.meshgrid(q,q)
        grid = np.stack([Q1,Q2], axis=-1).reshape((-1,2))
        return grid
        
    def generate_interpolation_nodes_3d(n):
        q = generate_CGL_nodes(n)
        Q1, Q2, Q3 = np.meshgrid(q,q,q)
        grid = np.stack([Q1,Q2,Q3], axis=-1).reshape((-1,3))
        return grid

    if(d==1):
      return generate_CGL_nodes(n).reshape(-1,1)
    elif(d==2):
      return generate_interpolation_nodes_2d(n)
    elif(d==3):
      return generate_interpolation_nodes_3d(n)
    else:
      print("Dimension d must be 1, 2 or 3")
      return

# Celle random

In [0]:
def generate_standard_cell(n, d, eps=0, theta=0):
    """
    Genera gli n^d punti di supporto per una cella standard
    eps --> distorsione applicata
    theta --> angoli di rotazione
    NOTA: se d==2 allora theta deve essere un numero oppure dafault==0, 
          se d==3 allora theta deve essere una lista di 3 numeri
    """
    assert n==2 or n==3 or n==4
    assert d==1 or d==2 or d==3

    #Distortion
    K = generate_interpolation_nodes(n,d)
    distortion = (np.random.random(K.shape)-.5)*eps
    K += distortion
    
    theta = np.array(theta)
    #2D rotation
    if(d==2):
      assert len(theta.shape)==0
      R = np.array([[np.cos(theta), np.sin(theta)],[-np.sin(theta), np.cos(theta)]])
      K = np.tensordot(K, R, axes=[[-1],[-1]])
    #3D rotation
    elif (d==3):
      assert theta.shape[0]==3
      a, b, c = theta[0], theta[1], theta[2]
      yaw = np.array([[np.cos(a),-np.sin(a),0.],[np.sin(a),np.cos(a),0.],[0.,0.,1.]])
      pitch = np.array([[np.cos(b),0.,np.sin(b)],[0.,1.,0.],[-np.sin(b),0.,np.cos(b)]])
      roll = np.array([[1.,0.,0.],[0.,np.cos(c),-np.sin(c)],[0.,np.sin(c),np.cos(c)]])
      R = np.matmul(np.matmul(yaw,pitch),roll)
      K = np.tensordot(K, R, axes=[[-1],[-1]])
      
    #Standardization
    m, M = np.min(K, axis=0), np.max(K, axis=0)
    h = M-m
    K -= m
    K /= h/2
    K -= 1
    return K

In [0]:
def plot_cell(K):
    """
    Restituisce uno scatterplot dei punti di supporto K
    NOTA: len(K.shape)==1 or K.shape[1]==2 or K.shape[1]==3
    """

    def plot_1d(K):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_xlim(-1.5,1.5)
        ax.set_ylim(0,10)
        xmin = -1.2
        xmax = 1.2
        y = 5
        height = 1
        plt.hlines(y, xmin, xmax)
        plt.vlines(xmin, y - height / 2., y + height / 2.)
        plt.vlines(xmax, y - height / 2., y + height / 2.)
        px = K
        plt.plot(K,y*np.ones(K.shape), 'ro', ms = 5, mfc = 'r')
        for x in K:
           plt.annotate(str(np.round(x[0],2)), (x[0],y), xytext = (x[0]+.09, y+.5),  horizontalalignment='right')
        plt.axis('off')
        return

    def plot_2d(K):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        scat = ax.scatter(K[...,0], K[...,1])
        return

    def plot_3d(K):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        scat = ax.scatter(K[...,0], K[...,1], K[...,2], c=K[...,2].flatten(), alpha=0.5)
        fig.colorbar(scat, shrink=0.5, aspect=5)
        return

    dim = K.shape
    assert dim[1]==1 or dim[1]==2 or dim[1]==3
    if(dim[1]==1):
      plot_1d(K)
      return
    elif(dim[1]==2):
      plot_2d(K)
      return
    elif(dim[1]==3):
      plot_3d(K)
      return

# Polinomi di Lagrange


In [0]:
def generate_L(K):
    """
    Restituisce la funzione L_K che mappa Khat in K.
    La funzione restituita è vettorizzata.
    Esempio:
      K = generate_standard_cell(3,2,.5,1)
      L = generate_L(K)
      L([[0,0]]) --> restituisce l'immagine del punto [0,0]
      L([[0,0],[5,2],[3,4]]) --> restituisce un array contenente le immagini dei tre punti

    """
    d = K.shape[1]
    n = int(pow(K.shape[0],1/d)+.5)
    Khat = generate_interpolation_nodes(n,d)
    q, w = gauss_lobatto(n, 10)
    nodes = np.array(q, dtype=float)

    def Lagrange(i,x):
        x = x.repeat(n-1).reshape(-1,d*(n-1))
        v = []
        for k in range(d):
            v.append(np.delete(nodes,np.where(nodes==Khat[i][k])))
        v = np.array(v).flatten()
        v = v.repeat(x.shape[0]).reshape((-1,len(v)), order='F')
        avoid = Khat[i].repeat(n-1).repeat(x.shape[0]).reshape(-1,d*(n-1), order='F')
        return np.prod((x-v)/(avoid-v), axis=1)
    
    def L(x):
        x = np.array(x)
        lgr = np.zeros((0,x.shape[0]))
        for i in range(len(Khat)):
            lgr = np.append(lgr, Lagrange(i,x))
        lgr = np.stack(lgr.reshape((-1,x.shape[0])),axis=-1)
        return np.matmul(lgr,K)

    return L

# Full inversion dataset


In [0]:
def dataset_full(n, d, distortions, angles, delta=.5, n_sampling_points=100):
    assert n==2 or n==3 or n==4
    assert d==2 or d==3
    distortions = np.array(distortions)
    angles = np.array(angles)
    if (d==3):
       assert angles.shape[1]==3

    xhat_1d = np.linspace(-delta-1,1+delta,n_sampling_points)
    if(d==2):
      xhat = np.stack(np.meshgrid(xhat_1d,xhat_1d), axis=-1).reshape((-1,d))
    else:
      xhat = np.stack(np.meshgrid(xhat_1d,xhat_1d,xhat_1d), axis=-1).reshape((-1,d))

    xtrain = np.zeros((0,(pow(n,d)+1)*d))
    ytrain = np.zeros((0,d))

    i=0
    start = timeit.default_timer()

    for A in distortions:
        i+=1
        clear_output(wait=True)
        for B in angles:
            

            K = generate_standard_cell(n,d,A,B)
            L = generate_L(K)
            x = L(xhat)
            mask = (x >= -1).all(axis=-1) * (x <= 1).all(axis=-1)
            x = x[mask,:]
            xhat = xhat[mask,:]
            
            xdata = np.concatenate((x, np.reshape(np.repeat(K.flatten(),x.shape[0]), (x.shape[0],-1) ,order='F')), axis=-1)

            xtrain = np.append(xtrain, xdata, axis=0)
            ytrain = np.append(ytrain, xhat[...,:], axis=0)

        stop = timeit.default_timer()
        time = np.round((stop-start)/60 , 2)
        wait = np.round((time/i)*(len(distortions)-i), 2)
        print("FULL INVERSE DATASET")
        print("Tempo trascorso: ", time, " minuti")
        print("Tempo rimanente: ", wait, " minuti")
        print("Percentuale di completamento: ", np.round((i/len(distortions))*100,2), "%") 


    return xtrain, ytrain

# Polynomial inversion dataset


In [0]:
from scipy.optimize import fsolve

def find_polynomial_inverse(K,up):
    d = K.shape[1]
    n = int(pow(K.shape[0],1/d)+.5) + up
    Khat = generate_interpolation_nodes(n,d)

    def residual(V):
        V = V.reshape((-1,d))
        F = generate_L(K)
        res = F(V)- Khat
        return res.flatten()
    
    return fsolve(residual,Khat.flatten()).reshape(-1,d)

In [0]:
def dataset_polynomial(n, up, d, distortions, angles):
    assert n==2 or n==3 or n==4
    assert d==2 or d==3
    distortions = np.array(distortions)
    angles = np.array(angles)
    if (d==3):
       assert angles.shape[1]==3

    xtrain = np.zeros((0,(pow(n,d)*d)))
    ytrain = np.zeros((0,(pow(n+up,d)*d)))

    i=0
    print("Percentuale di completamento: 0%")
    start = timeit.default_timer()


    for A in distortions:

        i+=1
        clear_output(wait=True)

        for B in angles:
            K = generate_standard_cell(n,d,A,B)
            xtrain = np.append(xtrain, K.flatten())
            ytrain = np.append(ytrain, find_polynomial_inverse(K,up))

        stop = timeit.default_timer()
        time = np.round((stop-start)/60 , 2)
        wait = np.round((time/i)*(len(distortions)-i), 2)
        print("POLYNOMIAL DATASET")
        print("Tempo trascorso: ", time, " minuti")
        print("Tempo rimanente: ", wait, " minuti")
        print("Percentuale di completamento: ", np.round((i/len(distortions))*100,2), "%")    

    xtrain = xtrain.reshape((-1,(pow(n,d)*d)))
    ytrain = ytrain.reshape((-1,(pow(n+up,d)*d)))

    return xtrain, ytrain